In [135]:
import math
import gym
import random
from gym import spaces
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import json
import datetime as dt
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

In [136]:
class Env(gym.Env):
    
    metadata = {'render.modes': ['human']}
    
    def __init__(self, row, col, num_players = 0, debug = False):
        super(Env, self).__init__()
        
        self.row = row
        self.col = col  
        self.current_step = 0
        self.bingo = np.array(random.sample(range(1,self.row*self.col+1), self.row*self.col))
        self.original_bingo_start_list = self.bingo.tolist()
        self.bingo = self.bingo.reshape(self.row,self.col)
        self.original_bingo_start = self.bingo
        self.bingo_state = np.zeros(self.row*self.col).reshape(self.row,self.col).astype('int32')
        self.bingo_selection_count = np.zeros(self.row*self.col).reshape(self.row,self.col).astype('int32')
        self.latest_number_cut = 0
        self.strikes = 0
        self.num_players = num_players
        self.player_cut_list = []
        self.action_space = spaces.Discrete(self.row*self.col)
        #self.action_space = spaces.Box(low=[0] , high=[row*col], dtype=np.int32)
        self.observation_space = spaces.Box(
          low=np.zeros(self.row*self.col).reshape(self.row,self.col), 
            high=np.ones(self.row*self.col).reshape(self.row,self.col), dtype=np.int32)
        self.debug = debug
        
    def reset(self):
               
        self.current_step = 0
        self.bingo = np.array(random.sample(range(1,self.row*self.col+1), self.row*self.col)).reshape(self.row,self.col)
        self.bingo_state = np.zeros(self.row*self.col).reshape(self.row,self.col).astype('int32')
        self.bingo_selection_count = np.zeros(self.row*self.col).reshape(self.row,self.col).astype('int32')
        self.strikes = 0
        
        return self._next_observation()
    
    def _next_observation(self):

        current_bingo_state = self.bingo_state   
        

        return np.array(current_bingo_state)
  
    def _take_action(self, action):
        
        self.current_step+=1
        self.latest_number_cut = self.bingo[math.floor(action/self.row),(action)%self.row]           
        is_number_cut = self.bingo_selection_count[math.floor(action/self.row),action%self.row]
    
        already_cut_reward = 0
        if(is_number_cut == 0):
            self.bingo[math.floor(action/self.row),(action)%self.row] = 0
            self.bingo_state[math.floor(action/self.row),action%self.row] = 1
            self.bingo_selection_count[math.floor(action/self.row),action%self.row] = 1
        else:
            self.bingo_selection_count[math.floor(action/self.row),action%self.row] += 1
            already_cut_reward = -self.bingo_selection_count[math.floor(action/self.row),action%self.row]          
        
        #For cuts
        cuts = 0
        for i in range(self.row):
            if(self.bingo_state[i,:].sum() == self.row):
                cuts+=1
        for i in range(self.col):
            if(self.bingo_state[:,i].sum() == self.col):
                cuts+=1
        
        diag1_sum = 0
        diag2_sum = 0
        
        for i in range(self.row):
            diag1_sum+= self.bingo_state[i,i]
            diag2_sum+= self.bingo_state[i,self.row-i-1]
            
        if(diag1_sum == self.row):            
            cuts+=1
        if(diag2_sum == self.row):            
            cuts+=1
    
        #bingo_dict = {0: "No_cut", 1: "B", 2: "BI", 3: "BIN", 4: "BING", 5: "BINGO", 6: "BINGO", 7: "BINGO"}
        
        reward_for_multiple_cuts = 0
        
        reward_for_multiple_cuts = cuts - self.strikes # +1 reward for a cut, + 2 reward for 2 cuts at the same time
        
        self.strikes = cuts
        
        total_add_reward = already_cut_reward + reward_for_multiple_cuts
                        
        return cuts, total_add_reward
    
    def step(self,action, mode = 'no_play'):
        
        reward = -1
        add_reward = 0
        
        if(mode == 'no_play'): # my turn i.e no players
            total_cuts,add_reward = self._take_action(action)
        else: # opponent's turn
            #tmp_list = list(self.original_bingo_start.reshape(self.row*self.col))
            #Using original because if player says the say number again there is an error
            player_cut_index = self.original_bingo_start_list.index(action)
            print("player_cut_index ", player_cut_index)
            total_cuts,add_reward = self._take_action(player_cut_index)
        
        reward+= add_reward
        
        if (total_cuts >= self.row):
            done = True
        else:
            done = False
        
        return self._next_observation(), reward, done, self.latest_number_cut, self.bingo, self.strikes
    
    def render(self, mode='train', close=False):
        # Render the environment to the screen
        
        if(mode=='train'):
            if(self.current_step > 500): #Just for check
                print(self.current_step)
            train_list.append([self.bingo, self.bingo_state, self.bingo_selection_count, "BINGO", self.strikes])
           # if (len(train_list) > 510):
            #    train_list = train_list[len(train_list) - 100 :]
        elif(mode == 'test'):
            test_list.append([self.bingo, self.bingo_state, self.bingo_selection_count, "BINGO", self.strikes])                
            #if (len(test_list) > 510):
             #   test_list = test_list[len(test_list) - 100 :]
            
        return

In [91]:
row = 5
col = 5

In [93]:

#bingo = np.array(random.sample(range(1,row*col+1), row*col)).reshape(row,col)
#bingo_state = np.zeros(row*col).reshape(row,col).astype('int32')
#bingo_selection_count = np.zeros(row*col).reshape(row,col).astype('int32')

bingo_env = Env(row, col, num_players = 0, debug = True)

model = PPO2(MlpPolicy, bingo_env,verbose = 0)
model.learn(total_timesteps=1)
model.save("deep_bingo")

In [99]:
del model
model = PPO2.load("deep_bingo_5lakh")

In [100]:
train_list= []
train_action_list= []
obs = bingo_env.reset()
done=False
while(done == False):
    action, _states = model.predict(obs)
    #print(action)
    obs, rewards, done, num_cut, bingo_curr, curr_strike = bingo_env.step(action)
    train_action_list.append(action)
    bingo_env.render(mode='train')
    #print(train_list[-1])

In [101]:
print(len(train_action_list))
train_list[-1]

16


[array([[ 0,  0, 21,  0,  0],
        [19,  0, 18,  0,  8],
        [ 6,  0,  0,  0,  5],
        [17,  0,  7,  0, 12],
        [ 0,  0,  0,  0,  0]]), array([[1, 1, 0, 1, 1],
        [0, 1, 0, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 0, 1, 0],
        [1, 1, 1, 1, 1]]), array([[1, 1, 0, 1, 1],
        [0, 1, 0, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 1, 0, 1, 0],
        [1, 1, 1, 1, 1]]), 'BINGO', 5]

In [103]:
for i in range(20):
    test_list = []
    test_action_list=[]
    obs = bingo_env.reset()
    done = False
    while(done ==False):
        action, _states = model.predict(obs)
        obs, rewards, done, num_cut, bingo_curr, curr_strike = bingo_env.step(action)
        test_action_list.append(action)
        bingo_env.render(mode='test')
    print(len(test_action_list))
    print(test_list[-1])

16
[array([[ 0,  0,  6,  0,  0],
       [10,  0, 11,  0,  3],
       [16,  0,  0,  0,  1],
       [ 2,  0, 20,  0,  4],
       [ 0,  0,  0,  0,  0]]), array([[1, 1, 0, 1, 1],
       [0, 1, 0, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 0, 1, 0],
       [1, 1, 1, 1, 1]]), array([[1, 1, 0, 1, 1],
       [0, 1, 0, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 0, 1, 0],
       [1, 1, 1, 1, 1]]), 'BINGO', 5]
16
[array([[ 0,  0,  2,  0,  0],
       [ 9,  0, 22,  0, 19],
       [ 7,  0,  0,  0, 18],
       [23,  0,  5,  0,  3],
       [ 0,  0,  0,  0,  0]]), array([[1, 1, 0, 1, 1],
       [0, 1, 0, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 0, 1, 0],
       [1, 1, 1, 1, 1]]), array([[1, 1, 0, 1, 1],
       [0, 1, 0, 1, 0],
       [0, 1, 1, 1, 0],
       [0, 1, 0, 1, 0],
       [1, 1, 1, 1, 1]]), 'BINGO', 5]
16
[array([[ 0,  0,  8,  0,  0],
       [ 4,  0,  5,  0, 16],
       [20,  0,  0,  0, 14],
       [18,  0,  3,  0,  2],
       [ 0,  0,  0,  0,  0]]), array([[1, 1, 0, 1, 1],
       [0, 

## Play

In [137]:
row = 5
col = 5
num_players_playing = int(input("Enter number of players "))
bingo_env_play = Env(row, col, num_players = num_players_playing, debug = True)
obs = bingo_env_play.reset()
done = False
player_win = False
counter = 0
strike_counter = 0

while(done ==False and player_win == False):
    #bingo_env_play.render(mode='play')
    for i in range(num_players_playing + 1):
        if(i != num_players_playing):
            msg = "Player " + str(i+1) + " Enter number you are cutting (Enter won/w if you have won): "
            pl_num = input(msg)
            if(pl_num == "won" or pl_num == "w"):
                print("Congrats! End of game")
                player_win = True
            else:
                pl_num = int(pl_num)
                bingo_env_play.step(pl_num,"play")
        else:
            action, _states = model.predict(obs)
            obs, rewards, done, info, curr_bingo, comp_strikes = bingo_env_play.step(action, "no_play")
            if(done == True):
                print("I won. Game over")
            else:
                print("Number I am striking is ", info)
                print(curr_bingo)
                print("")
                if(strike_counter != comp_strikes):
                    print("I have ", comp_strikes, "strike(s)")
                strike_counter = comp_strikes
        

Enter number of players 1
Player 1 Enter number you are cutting (Enter won/w if you have won): 12
player_cut_index  20
Number I am striking is  5
[[19  7 14 17  4]
 [21  8  2  9 11]
 [ 1  3 24 10 16]
 [ 6 18 15 12 20]
 [ 0 22 25 23  0]]

Player 1 Enter number you are cutting (Enter won/w if you have won): 19
player_cut_index  18
Number I am striking is  4
[[19  7 14 17  0]
 [21  8  2  9 11]
 [ 1  3 24 10 16]
 [ 6 18 15  0 20]
 [ 0 22 25 23  0]]

Player 1 Enter number you are cutting (Enter won/w if you have won): w
Congrats! End of game
Number I am striking is  24
[[19  7 14 17  0]
 [21  8  2  9 11]
 [ 1  3  0 10 16]
 [ 6 18 15  0 20]
 [ 0 22 25 23  0]]



In [115]:

aaa.index(3)

2

In [130]:
aaa = np.array([1,2,3,4,5,6,7,8,9]).reshape(3,3)
print(aaa)
aaa.reshape(9,)
print(aaa)
bbb = list(aaa)
print(bbb)
bbb.index(7)

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[1 2 3]
 [4 5 6]
 [7 8 9]]
[array([1, 2, 3]), array([4, 5, 6]), array([7, 8, 9])]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
index()